In [1]:
import requests

page = requests.get("http://dataquestio.github.io/web-scraping-pages/simple.html")
page

<Response [200]>

In [2]:
page.status_code

200

In [3]:
page.content

b'<!DOCTYPE html>\n<html>\n    <head>\n        <title>A simple example page</title>\n    </head>\n    <body>\n        <p>Here is some simple content for this page.</p>\n    </body>\n</html>'

# Parsing a page with BeautifulSoup

In [9]:
from bs4 import BeautifulSoup

In [10]:
soup = BeautifulSoup(page.content, 'html.parser')

In [11]:
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


In [12]:
list(soup.children)

['html', '\n', <html>
 <head>
 <title>A simple example page</title>
 </head>
 <body>
 <p>Here is some simple content for this page.</p>
 </body>
 </html>]

In [13]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

In [15]:
html = list(soup.children)[2]

In [16]:
list(html.children)

['\n', <head>
 <title>A simple example page</title>
 </head>, '\n', <body>
 <p>Here is some simple content for this page.</p>
 </body>, '\n']

In [17]:
body = list(html.children)[3]

In [18]:
list(body.children)

['\n', <p>Here is some simple content for this page.</p>, '\n']

In [19]:
p = list(body.children)[1]

In [20]:
p.get_text()

'Here is some simple content for this page.'

# Finding all instances of a tag at once

In [21]:
soup = BeautifulSoup(page.content, 'html.parser')
soup.find_all('p')

[<p>Here is some simple content for this page.</p>]

In [25]:
soup.find_all('p')[0].get_text()

'Here is some simple content for this page.'

In [26]:
soup.find('p')

<p>Here is some simple content for this page.</p>

# Searching for tags by class and id

In [28]:
page = requests.get("http://dataquestio.github.io/web-scraping-pages/ids_and_classes.html")
soup = BeautifulSoup(page.content, 'html.parser')
soup

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                First paragraph.
            </p>
<p class="inner-text">
                Second paragraph.
            </p>
</div>
<p class="outer-text first-item" id="second">
<b>
                First outer paragraph.
            </b>
</p>
<p class="outer-text">
<b>
                Second outer paragraph.
            </b>
</p>
</body>
</html>

In [30]:
soup.find_all('p', class_='outer-text')

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>, <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [31]:
soup.find_all(class_='outer-text')

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>, <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [32]:
soup.find_all(id='first')

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>]

# Using CSS Selectors

In [33]:
soup.select("div p")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>, <p class="inner-text">
                 Second paragraph.
             </p>]

# Downloading Weather data

In [37]:
page = requests.get("http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_='tombstone-container')
tonight = forecast_items[1]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Tonight: A 20 percent chance of showers and thunderstorms before 11pm.  Mostly cloudy, with a low around 60. West wind 9 to 16 mph, with gusts as high as 22 mph. " class="forecast-icon" src="DualImage.php?i=hi_ntsra&amp;j=nbkn&amp;ip=20" title="Tonight: A 20 percent chance of showers and thunderstorms before 11pm.  Mostly cloudy, with a low around 60. West wind 9 to 16 mph, with gusts as high as 22 mph. "/>
 </p>
 <p class="short-desc">
  Slight Chance
  <br/>
  T-storms then
  <br/>
  Mostly Cloudy
 </p>
 <p class="temp temp-low">
  Low: 60 °F
 </p>
</div>


# Extracting information from the page

In [39]:
period = tonight.find(class_ = 'period-name').get_text()
short_desc = tonight.find(class_='short-desc').get_text()
temp = tonight.find(class_ = 'temp').get_text()
print(period)
print(short_desc)
print(temp)

Tonight
Slight ChanceT-storms thenMostly Cloudy
Low: 60 °F


In [41]:
img = tonight.find('img')
desc = img['title']
print(desc)

Tonight: A 20 percent chance of showers and thunderstorms before 11pm.  Mostly cloudy, with a low around 60. West wind 9 to 16 mph, with gusts as high as 22 mph. 


# Extracting all the information from the page

In [47]:
period_tags = seven_day.select(".tombstone-container, .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['\nToday\nSlight ChanceT-storms andPatchy FogHigh: 72 °F',
 'Today',
 '\nTonight\nSlight ChanceT-storms thenMostly CloudyLow: 60 °F',
 'Tonight',
 '\nThursday\nDecreasingCloudsHigh: 71 °F',
 'Thursday',
 '\nThursdayNight\nPartly CloudyLow: 59 °F',
 'ThursdayNight',
 '\nFriday\nMostly SunnyHigh: 71 °F',
 'Friday',
 '\nFridayNight\nPartly CloudyLow: 59 °F',
 'FridayNight',
 '\nSaturday\nMostly SunnyHigh: 70 °F',
 'Saturday',
 '\nSaturdayNight\nPartly Cloudyand Breezythen PartlyCloudyLow: 59 °F',
 'SaturdayNight',
 '\nSunday\nMostly SunnyHigh: 71 °F',
 'Sunday']

In [45]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['Slight ChanceT-storms andPatchy Fog', 'Slight ChanceT-storms thenMostly Cloudy', 'DecreasingClouds', 'Partly Cloudy', 'Mostly Sunny', 'Partly Cloudy', 'Mostly Sunny', 'Partly Cloudyand Breezythen PartlyCloudy', 'Mostly Sunny']
['High: 72 °F', 'Low: 60 °F', 'High: 71 °F', 'Low: 59 °F', 'High: 71 °F', 'Low: 59 °F', 'High: 70 °F', 'Low: 59 °F', 'High: 71 °F']
['Today: A 20 percent chance of showers and thunderstorms after 11am.  Patchy fog before 11am.  Otherwise, mostly sunny, with a high near 72. West southwest wind 7 to 15 mph, with gusts as high as 20 mph. ', 'Tonight: A 20 percent chance of showers and thunderstorms before 11pm.  Mostly cloudy, with a low around 60. West wind 9 to 16 mph, with gusts as high as 22 mph. ', 'Thursday: Mostly cloudy, then gradually becoming sunny, with a high near 71. West wind 9 to 16 mph, with gusts as high as 21 mph. ', 'Thursday Night: Partly cloudy, with a low around 59. West wind 10 to 16 mph, with gusts as high as 21 mph. ', 'Friday: Mostly sunn

# Combining our data into a Pandas Dataframe

In [59]:
import pandas as pd
weather = pd.DataFrame({
    #"period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc":descs
})
weather

,short_desc,temp,desc
0,Slight ChanceT-storms andPatchy Fog,High: 72 °F,Today: A 20 percent chance of showers and thun...
1,Slight ChanceT-storms thenMostly Cloudy,Low: 60 °F,Tonight: A 20 percent chance of showers and th...
2,DecreasingClouds,High: 71 °F,"Thursday: Mostly cloudy, then gradually becomi..."
3,Partly Cloudy,Low: 59 °F,"Thursday Night: Partly cloudy, with a low arou..."
4,Mostly Sunny,High: 71 °F,"Friday: Mostly sunny, with a high near 71. Wes..."
5,Partly Cloudy,Low: 59 °F,"Friday Night: Partly cloudy, with a low around..."
6,Mostly Sunny,High: 70 °F,"Saturday: Mostly sunny, with a high near 70."
7,Partly Cloudyand Breezythen PartlyCloudy,Low: 59 °F,"Saturday Night: Partly cloudy, with a low arou..."
8,Mostly Sunny,High: 71 °F,"Sunday: Mostly sunny, with a high near 71."


In [60]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    72
1    60
2    71
3    59
4    71
5    59
6    70
7    59
8    71
Name: temp_num, dtype: object

In [61]:
weather["temp_num"].mean()

65.77777777777777

In [62]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0    False
1     True
2    False
3     True
4    False
5     True
6    False
7     True
8    False
Name: temp, dtype: bool

In [64]:
weather[is_night]

,short_desc,temp,desc,temp_num,is_night
1,Slight ChanceT-storms thenMostly Cloudy,Low: 60 °F,Tonight: A 20 percent chance of showers and th...,60,True
3,Partly Cloudy,Low: 59 °F,"Thursday Night: Partly cloudy, with a low arou...",59,True
5,Partly Cloudy,Low: 59 °F,"Friday Night: Partly cloudy, with a low around...",59,True
7,Partly Cloudyand Breezythen PartlyCloudy,Low: 59 °F,"Saturday Night: Partly cloudy, with a low arou...",59,True
